In [0]:
external_azure_path = 'abfss://container20260122@datalake20260121.dfs.core.windows.net/'

dbutils.fs.ls(f'{external_azure_path}/02-silver')

In [0]:
# Creating silver tmp tables to create gold analytic data
for delta_table in dbutils.fs.ls(f'{external_azure_path}/02-silver'):
    (spark.read
        .format("delta")
        .load(delta_table.path)
        .createOrReplaceTempView(f'tmp_silver_{delta_table.name.rstrip('/')}')
    )

In [0]:
%sql
DESCRIBE tmp_silver_orders

In [0]:
%py
df_gold_ny_sales = spark.sql('''
    -- Creating SQL query to aggregate order data                            
    SELECT 
          shipped_date
        , ROUND(SUM(total_sales), 2) AS ny_total_sales
    FROM
        tmp_silver_orders
    WHERE
        store_state         = 'NY'
        AND status          = 'Delivered' 
        AND shipped_date    IS NOT NULL
    GROUP BY 
        shipped_date                       
''')

display(df_gold_ny_sales)

In [0]:
# Writing on silver azure external storage in delta format
(df_gold_ny_sales.write
    .mode('overwrite')
    .format('delta')
    .option('mergeSchema', 'true')
    .save(f'{external_azure_path}/03-gold/ny-sales')
)

In [0]:
%sql
-- Creating a new schema to store gold tables
CREATE SCHEMA IF NOT EXISTS bike_store.gold_schema

In [0]:
%sql
DROP TABLE IF EXISTS bike_store.gold_schema.ny_sales

In [0]:
%sql
-- Creating databricks delta table
CREATE TABLE IF NOT EXISTS bike_store.gold_schema.ny_sales
LOCATION 'abfss://container20260122@datalake20260121.dfs.core.windows.net/03-gold/ny-sales'